### Simple examples
(Диффуры которые легко интерпретировать)

In [1]:
import torch
import pyro
from pyro.infer import MCMC, NUTS

In [2]:
import os
import sys
import inspect

# insert root dir into sys
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
root_dir = os.path.dirname(currentdir)
# print(root_dir)

if root_dir not in sys.path:
     sys.path.insert(0, root_dir)

In [3]:
import sim5_hw as sm

In [4]:
# efficiency matrix:
U = 0.1*torch.ones((8, 8))
U[0][:3] = torch.tensor([0.7, 0.8, 0.3])
U[1][2:] = torch.tensor([0.3, 0.5, 0.9, 0.9, 0.9, 0.2])
U[2][:4] = torch.tensor([0.9, 0.9, 0.5, 0.2])
U[3][2:-1] = torch.tensor([0.9, 0.5, 0.7, 0.7, 0.7])
U[4][:] = torch.tensor([0.6, 0.7, 0.9, 0.9, 0.5, 0.5, 0.5, 0.2])
U[5][:] = torch.tensor([0.9, 0.9, 0.6, 0.4, 0.1, 0.1, 0.1, 0.1])
U[6][4:] = torch.tensor([0.9, 0.9, 0.9, 0.5])
U[7][4:] = torch.tensor([0.9, 0.9, 0.9, 0.7])


Model specification:

In [9]:
    A_spec = {
        "decision_matrix": None,
        "units": {

            # needed only if there is neither types nor counts given:
            "possible_types": [0, 1, 2, 3, 4],
            "chosen_count": 3,
            # "types": [0, 1, 4],

            # "counts": torch.tensor([3, 3, 4]).type(torch.float)
            # maximal amount of units to sample for any type
            # must be given if "counts" not: 
            "max_count": 4,
        },
        "goal": lambda x, y: (y <= 0).all(),
        ## "goal": lambda x, y: y[1] <= 0 and y[2] <= 0
        # "goal": lambda x, y: y[0] <= 0 and y[1] <= 0 and x[-1] > 2
    }
    B_spec = {
        "decision_matrix": None,
        "units": {
            "types": [1, 2, 3],
            "counts": torch.tensor([1, 4,  4]).type(torch.float)
        },
        "goal": lambda x, y: (x <= 0).all() and y[2] >= 2
        # "goal": lambda x, y: x[1] <= 0 and x[-1] <= 0,  # and y[2] >= 2
    }

    sim_spec = {
        "agents": {"A": A_spec, "B": B_spec},

        "T": 30,
        "dt": 0.5,
        # "dt": 0.5,
        "init_factor": -110.0,

        "U": U,

        # choose side A (i.e. want A to win):
        "scores": [
            {
                "test": lambda goalA, goalB: goalA and not goalB,
                "once": False,
                "factor": 0,
                # will be exited once happend, factor been be overriden
                "exit": True
            },
            {
                "test": lambda goalA, goalB: not goalA and goalB,
                "once": False,
                "factor": -1000,
                # will be exited once happend, factor been be overriden
                "exit": True
            },
            {
                "test": lambda goalA, goalB: not goalA and not goalB,
                # only once from all times will factor been used  
                "once": True,
                "factor": 10,
                "exit": False
            }
        ] 
    }


In [10]:
mcmc, losses = sm.test_mcmc(30, sim_spec, mdbg=False, edbg=False)

Sample: 100%|██████████| 60/60 [03:27,  3.46s/it, step size=2.16e+01, acc. prob=0.299]


Collect result (solution):

In [7]:
sim_spec = sm.update_spec(sim_spec, mcmc, idx=-1, side="A", dbg=False)
sim_spec['agents']['A']

{'decision_matrix': tensor([[0.3847, 0.1720, 0.1005],
         [0.4989, 0.5718, 0.3257],
         [0.1165, 0.2562, 0.5738]]),
 'goal': <function __main__.<lambda>>,
 'units': {'chosen_count': 3,
  'counts': tensor([1.9006, 3.9219, 1.7175]),
  'max_count': 4,
  'possible_types': [0, 1, 2, 3, 4],
  'types': (0, 1, 4)}}

Collect factor (error) and others params:

In [14]:
losses = []
ehandler = sm.test_mk_model(sim_spec, losses, False, False)

sm.show_model_last_state_context(
    ehandler, ["x", "y", "Ua", "Ub", "Ax", "Ay", "events_to_check"])
sm.show_model_last_state_context(
    ehandler, ["utypes"],
    observation_name="sample_space_with_size_a_utype_sample_space_size")
sm.show_model_trace(ehandler, ["x0", "Ax_T", "y0", "Ay_T"])




observations state_context::

x
tensor([0., 0., 0.])

y
tensor([0.3802, 2.6238, 1.8530])

Ua
tensor([[0.8000, 0.1000, 0.7000],
        [0.3000, 0.3000, 0.9000],
        [0.1000, 0.5000, 0.9000]])

Ub
tensor([[0.1000, 0.9000, 0.1000],
        [0.1000, 0.9000, 0.1000],
        [0.9000, 0.1000, 0.7000]])

Ax
tensor([[0.1764, 0.3226, 0.2039],
        [0.3414, 0.2521, 0.3723],
        [0.4822, 0.4253, 0.4238]])

Ay
tensor([[0.5553, 0.4974, 0.0142],
        [0.1019, 0.2129, 0.0728],
        [0.3428, 0.2897, 0.9130]])

events_to_check
OrderedDict([('goalA', tensor(False)), ('goalB', tensor(False)), ('goalAoverB', False), ('exit?', None)])

observations state_context::
no observations to show

trace:

event_error_factor:
tensor(-100.)
